We will analyze a dataset containing information about flights in the USA that stems from the [Bureau of Transportation Statistics](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time). In the zip file provided, you will find a directory *airline_data* containing the files 2016_1.csv, ..., 2016_6.csv. Each line of a file (except the headers) contains the flight date, the airline ID, the flight number, the origin airport, the destination airport, the departure time, the departure delay in minutes, the arrival time, the arrival delay in minutes, the time in air in minutes, and the distance between both airports in miles. 

Preparation (if not done already):

1. Copy all the csv files to your virtual machine (e.g., via scp). Alternatively, you can also log in to the virtual machine and download the zip file directy from Absalon.
2. Create a directory 'airline_data' on your local Hadoop cluster. Afterwards, copy the csv files from your virtual machine to the Hadoop cluster via 'hadoop fs -put airline_data/*.csv airline_data/'

In [1]:
# generate an RDD based on all the csv files given in the airline_data directory.
# MAKE SURE THAT ONLY THE 2016_*.csv FILES ARE GIVEN IN THE DIRECTORY
airline_data = sc.textFile ("hdfs:///user/lsda/airline_data/*.csv")

In [2]:
# each csv file contains a header describing the data
header = airline_data.first()
print("Header information given in the first csv file:\n\n{}".format(header))

Header information given in the first csv file:

"FL_DATE","AIRLINE_ID","FL_NUM","ORIGIN","DEST","DEP_TIME","DEP_DELAY","ARR_TIME","ARR_DELAY","AIR_TIME","DISTANCE",


In [3]:
# filter the RDD to remove this header information (each csv file 
# contains such a line)
airline_data = airline_data.filter(lambda line: line != header)

# get the first 10 elements and print them
print("First 10 elements of the RDD:")
airline_data.take(10)

First 10 elements of the RDD:


['2016-01-01,19790,"1248","DTW","LAX","1935",0.00,"2120",-24.00,249.00,1979.00,',
 '2016-01-01,19790,"1251","ATL","GRR","2130",5.00,"2319",-2.00,92.00,640.00,',
 '2016-01-01,19790,"1254","LAX","ATL","2256",1.00,"0547",-13.00,207.00,1947.00,',
 '2016-01-01,19790,"1255","SLC","ATL","1700",4.00,"2213",-16.00,173.00,1590.00,',
 '2016-01-01,19790,"1256","BZN","MSP","1012",72.00,"1420",124.00,121.00,874.00,',
 '2016-01-01,19790,"1257","ATL","BNA","1356",83.00,"1402",83.00,38.00,214.00,',
 '2016-01-01,19790,"1257","BNA","ATL","1446",86.00,"1644",74.00,37.00,214.00,',
 '2016-01-01,19790,"1258","ATL","JAX","0946",1.00,"1053",3.00,45.00,270.00,',
 '2016-01-01,19790,"1258","JAX","ATL","1144",-1.00,"1247",-15.00,43.00,270.00,',
 '2016-01-01,19790,"1259","ATL","OKC","2107",-3.00,"2224",-12.00,116.00,761.00,']

In [4]:
def parse(line):
    
    line = line.split(',')
    
    try:
    
        airline_id = line[1]
        origin = line[3].strip('\"')
        dest = line[4].strip('\"')
        dep_delay = float(line[6])
        arr_delay = float(line[8])
        distance = float(line[10])
        
        return (airline_id, origin, dest, dep_delay, arr_delay, distance)
    
    except Exception as e:
        
        # in case of an error: simply return 'None'
        return None

In [5]:
# apply the parsing function to each element via the map
# transformation; afterwards, remove all elements that
# could not be parsed properly.
airlines = airline_data.map(parse)
airlines = airlines.filter(lambda line: line is not None)

In [6]:
# let's inspect the first ten elements
airlines.take(10)

[('19790', 'DTW', 'LAX', 0.0, -24.0, 1979.0),
 ('19790', 'ATL', 'GRR', 5.0, -2.0, 640.0),
 ('19790', 'LAX', 'ATL', 1.0, -13.0, 1947.0),
 ('19790', 'SLC', 'ATL', 4.0, -16.0, 1590.0),
 ('19790', 'BZN', 'MSP', 72.0, 124.0, 874.0),
 ('19790', 'ATL', 'BNA', 83.0, 83.0, 214.0),
 ('19790', 'BNA', 'ATL', 86.0, 74.0, 214.0),
 ('19790', 'ATL', 'JAX', 1.0, 3.0, 270.0),
 ('19790', 'JAX', 'ATL', -1.0, -15.0, 270.0),
 ('19790', 'ATL', 'OKC', -3.0, -12.0, 761.0)]

In [7]:
# (a) Shortest Flight Distance
flights = airlines.map(lambda line: (line[0], line[5]))
results_dist = flights.reduceByKey(lambda a, b: min(a,b))
print("ID\tDistance")
for result in results_dist.collect():
    print(result)

ID	Distance
('20366', 69.0)
('20409', 68.0)
('19393', 137.0)
('20304', 30.0)
('19690', 84.0)
('20416', 177.0)
('19790', 94.0)
('19805', 83.0)
('19930', 31.0)
('21171', 236.0)
('20436', 332.0)
('19977', 108.0)


In [8]:
# (b) Late Arrival Counts
delayed_flights = airlines.filter(lambda line: line[4] > 0)
delayed = delayed_flights.map(lambda line: (line[0],1))
results_delayed = delayed.reduceByKey(lambda a,b: a+b)

flights = airlines.map(lambda line: (line[0],1))
results = flights.reduceByKey(lambda a,b: a+b)

results_combined = results_delayed.join(results)
results = results_combined.map(lambda line: (line[0],(line[1][0],line[1][1],line[1][0]/line[1][1]*100))).collect()

print("ID\tDelayed\tTotal\tPercentage")
for result in results:
    print(result)

ID	Delayed	Total	Percentage
('20366', (77122, 244919, 31.488777922496823))
('20409', (55437, 137534, 40.30785114953394))
('19393', (234624, 633348, 37.04503685177817))
('20304', (97456, 293875, 33.1623989791578))
('19690', (12158, 37800, 32.164021164021165))
('20416', (31417, 65913, 47.664345425029964))
('19790', (117601, 452072, 26.013776566564616))
('19805', (165059, 452040, 36.514246526856034))
('19930', (26422, 87496, 30.19795190637286))
('21171', (14084, 32891, 42.82022437748928))
('20436', (12978, 43444, 29.872939876622777))
('19977', (69094, 254466, 27.152546902140166))


In [9]:
# (c) Mean and Standard Deviation for Arrival Delays
import math
means = airlines.map(lambda line: (line[0],(line[4],1)))
means = means.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])).mapValues(lambda x: x[0]/x[1])

# calculate variance / get standard deviation from it
deviation = airlines.map(lambda line: (line[0],line[4]))
deviation = deviation.mapValues(lambda x: (1, x, x*x)).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1], x[2]+y[2]))
deviation = deviation.mapValues(lambda x: (math.sqrt(x[2]/x[0] - (x[1]/x[0])**2)))

results = means.join(deviation).collect()

print("ID\tMean\tDeviation")
for result in results:
    print(result)

ID	Mean	Deviation
('20366', (3.2057047432008132, 44.650629511539684))
('20409', (8.245044861634215, 44.66785397812182))
('19393', (3.227560519651124, 30.30109303783044))
('20304', (3.9044287537218203, 44.122724012248916))
('19690', (-0.8930423280423281, 23.126745648474056))
('20416', (12.206544991124664, 42.44123916637707))
('19790', (-1.4154625811817587, 39.86619669104993))
('19805', (4.480922042297141, 44.625990837252665))
('19930', (-3.122085581055134, 26.807509492056))
('21171', (7.240521723267763, 37.55520264018218))
('20436', (1.7378694411196023, 46.45093455357346))
('19977', (-0.19219463503965167, 42.873551097034806))


In [10]:
# (d) Top-10 of Arrival Delays
flight_delays = airlines.map(lambda line: (line[0],[line[4]]))
#get airline id and delays as a list
flight_delays_byId = flight_delays.reduceByKey(lambda x,y: x+y).sortByKey(True)
#sort the list
flight_delays_byId_sorted=flight_delays_byId.map( lambda x:(x[0],sorted( x[1],reverse=True)[:10] ) )

results=flight_delays_byId_sorted.collect()

print("ID\tDelays")
for result in results:
    print(result)

ID	Delays
('19393', [748.0, 640.0, 635.0, 625.0, 594.0, 581.0, 579.0, 561.0, 560.0, 543.0])
('19690', [1198.0, 1066.0, 995.0, 958.0, 949.0, 924.0, 911.0, 868.0, 808.0, 757.0])
('19790', [1231.0, 1223.0, 1185.0, 1176.0, 1156.0, 1153.0, 1153.0, 1150.0, 1143.0, 1138.0])
('19805', [1660.0, 1659.0, 1568.0, 1557.0, 1551.0, 1543.0, 1533.0, 1523.0, 1495.0, 1493.0])
('19930', [993.0, 890.0, 876.0, 635.0, 621.0, 567.0, 535.0, 514.0, 505.0, 504.0])
('19977', [1213.0, 1120.0, 1110.0, 1107.0, 1025.0, 1019.0, 993.0, 990.0, 990.0, 968.0])
('20304', [1370.0, 1320.0, 1316.0, 1281.0, 1258.0, 1254.0, 1235.0, 1228.0, 1221.0, 1217.0])
('20366', [1237.0, 1234.0, 1191.0, 1187.0, 1175.0, 1158.0, 1158.0, 1152.0, 1137.0, 1125.0])
('20409', [798.0, 765.0, 746.0, 676.0, 659.0, 637.0, 616.0, 611.0, 587.0, 577.0])
('20416', [892.0, 706.0, 602.0, 584.0, 581.0, 550.0, 542.0, 541.0, 531.0, 501.0])
('20436', [707.0, 680.0, 645.0, 630.0, 629.0, 623.0, 597.0, 563.0, 556.0, 556.0])
('21171', [391.0, 389.0, 381.0, 364.0, 3